# Analysis of static Vicon and EKF noise characteristics 

In [8]:
%load_ext autoreload
%autoreload 2

import scipy
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.spatial.transform import Rotation as R
import loading_utils as lu
import analysis_utils as au
import plotting_utils as pu

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# Data Loading
state_csv = Path("~/catkin_ws/ros_logs/state_log.csv")
mocap_csv = Path("~/catkin_ws/ros_logs/mocap_log.csv")
state_estimates = lu.QuadStateEstimates.from_csv(state_csv)
vicon_measurements = lu.ViconMeasurements.from_csv(mocap_csv)

# Check for unsorted timestamps and print the number of violations and the states
unsorted = np.diff(vicon_measurements.time) <= 0
indices = np.where(unsorted)[0] + 1  # +1 to get the index of the unsorted element
print("Number of Vicon timestamp violations:", np.sum(unsorted))
print("Vicon states with timestamp violations:\n", vicon_measurements.df.iloc[indices])

Number of Vicon timestamp violations: 6289
Vicon states with timestamp violations:
        header_stamp  header_seq header_frame     pos_x     pos_y     pos_z  \
59     5.536884e+07       30107        world -0.040651  0.276753  0.105203   
61     1.755369e+09       30108        world -0.040652  0.276763  0.105220   
63     1.755369e+09       30109        world -0.040649  0.276756  0.105206   
65     1.755369e+09       30110        world -0.040648  0.276775  0.105217   
67     1.755369e+09       30111        world -0.040628  0.276770  0.105223   
...             ...         ...          ...       ...       ...       ...   
14698  1.755369e+09       38564        world -0.040654  0.276760  0.105201   
14700  1.755369e+09       38565        world -0.040663  0.276755  0.105214   
14702  1.755369e+09       38566        world -0.040666  0.276756  0.105213   
14704  1.755369e+09       38567        world -0.040651  0.276759  0.105211   
14788  1.755369e+09       38651        world -0.040649  0.

# Vicon Noise Analysis

This section analyzes the noise characteristics of the Vicon motion capture system measurements. We will explore the data, visualize the noise, and discuss its implications for state estimation and sensor fusion.

**Contents:**
- Statistical analysis of noise
- Visualization of position and orientation noise

In [3]:
# --- Position Analysis ---
vicon_stats = au.position_analysis(vicon_measurements.position.to_numpy())
au.print_position_analysis(vicon_stats, label="Vicon Position")

Vicon Position Mean:
 X: -0.0407;    Y: 0.2768;    Z: 0.1052
Vicon Position Std Dev:
 X: 0.00000828;    Y: 0.00000717;    Z: 0.00000838
Vicon Position 95th Percentile:
 X: 0.00001372;    Y: 0.00001240;    Z: 0.00001396
Vicon Position 99th Percentile:
 X: 0.00001935;    Y: 0.00001726;    Z: 0.00001991
Vicon Position Max Deviation:
 X: 0.00003198;    Y: 0.00002725;    Z: 0.00003526
Vicon Position RMS:
 X: 0.00000828;    Y: 0.00000717;    Z: 0.00000838
Vicon Position Covariance Matrix (rows/cols: X, Y, Z):
         X           Y           Z
X  6.8490e-11  2.5619e-11  3.0473e-11
Y  2.5619e-11  5.1474e-11  2.3879e-11
Z  3.0473e-11  2.3879e-11  7.0232e-11


In [4]:
# --- Orientation Analysis ---
vicon_R = R.from_quat(vicon_measurements.orientation, scalar_first=True)
vicon_R_stats = au.orientation_analysis(vicon_R)
au.print_orientation_analysis(vicon_R_stats, label="Vicon Orientation")

Vicon Orientation Mean (as quaternion):
 W: 0.99799;    X: -0.00304;    Y: 0.01855;    Z: 0.06051
Vicon Orientation Std Dev (as deg):
 X: 0.00431272;    Y: 0.00445195;    Z: 0.00213834
Vicon Orientation Vector RMS (as deg):
 X: 0.00431272;    Y: 0.00445195;    Z: 0.00213834
Vicon Orientation Covariance Matrix (rows/cols: x, y, z):
         x           y           z
x  5.6661e-09  -4.8490e-09  2.8372e-10
y  -4.8490e-09  6.0379e-09  -1.2279e-09
z  2.8372e-10  -1.2279e-09  1.3930e-09


# EKF Noise Analysis

In [5]:
# --- Position Analysis ---
ekf_stats = au.position_analysis(state_estimates.position.to_numpy())
au.print_position_analysis(ekf_stats, label="EKF Position")


EKF Position Mean:
 X: -0.0407;    Y: 0.2768;    Z: 0.1052
EKF Position Std Dev:
 X: 0.00000697;    Y: 0.00000611;    Z: 0.00000860
EKF Position 95th Percentile:
 X: 0.00001165;    Y: 0.00001026;    Z: 0.00001482
EKF Position 99th Percentile:
 X: 0.00001588;    Y: 0.00001438;    Z: 0.00002098
EKF Position Max Deviation:
 X: 0.00002370;    Y: 0.00001942;    Z: 0.00003191
EKF Position RMS:
 X: 0.00000697;    Y: 0.00000611;    Z: 0.00000860
EKF Position Covariance Matrix (rows/cols: X, Y, Z):
         X           Y           Z
X  4.8541e-11  1.4021e-11  2.1938e-11
Y  1.4021e-11  3.7284e-11  1.0543e-11
Z  2.1938e-11  1.0543e-11  7.3908e-11


In [6]:
# --- Orientation Analysis ---
ekf_R = R.from_quat(state_estimates.orientation, scalar_first=True)
ekf_R_stats = au.orientation_analysis(ekf_R)
au.print_orientation_analysis(ekf_R_stats, label="EKF Orientation")

EKF Orientation Mean (as quaternion):
 W: 0.99799;    X: -0.00304;    Y: 0.01855;    Z: 0.06051
EKF Orientation Std Dev (as deg):
 X: 0.00498905;    Y: 0.00511241;    Z: 0.00251177
EKF Orientation Vector RMS (as deg):
 X: 0.00498905;    Y: 0.00511241;    Z: 0.00251177
EKF Orientation Covariance Matrix (rows/cols: x, y, z):
         x           y           z
x  7.5837e-09  -6.3500e-09  2.4887e-10
y  -6.3500e-09  7.9634e-09  -1.6116e-09
z  2.4887e-10  -1.6116e-09  1.9222e-09


In [7]:
# --- Linear Velocity Analysis ---
# Given that the Quad is static, we can analyze the velocity estimates as they were position measurements.
ekf_vel_stats = au.position_analysis(state_estimates.velocity_linear.to_numpy())
au.print_position_analysis(ekf_vel_stats, label="EKF Velocity")

EKF Velocity Mean:
 X: -0.0000;    Y: 0.0000;    Z: 0.0000
EKF Velocity Std Dev:
 X: 0.00023545;    Y: 0.00020985;    Z: 0.00030518
EKF Velocity 95th Percentile:
 X: 0.00038688;    Y: 0.00035727;    Z: 0.00051369
EKF Velocity 99th Percentile:
 X: 0.00052885;    Y: 0.00046881;    Z: 0.00071180
EKF Velocity Max Deviation:
 X: 0.00089310;    Y: 0.00071925;    Z: 0.00108899
EKF Velocity RMS:
 X: 0.00023545;    Y: 0.00020985;    Z: 0.00030518
EKF Velocity Covariance Matrix (rows/cols: X, Y, Z):
         X           Y           Z
X  5.5450e-08  1.9056e-08  2.8925e-08
Y  1.9056e-08  4.4045e-08  9.3775e-09
Z  2.8925e-08  9.3775e-09  9.3157e-08


In [8]:
# --- Linear Acceleration Analysis ---
# Given that the Quad is static, we can analyze the linear acceleration estimates as they were position measurements.
ekf_acc_stats = au.position_analysis(state_estimates.acceleration_linear.to_numpy())
au.print_position_analysis(ekf_acc_stats, label="EKF Linear Acceleration")

EKF Linear Acceleration Mean:
 X: -0.0000;    Y: 0.0000;    Z: 0.0000
EKF Linear Acceleration Std Dev:
 X: 0.00184113;    Y: 0.00164473;    Z: 0.00239222
EKF Linear Acceleration 95th Percentile:
 X: 0.00306998;    Y: 0.00276120;    Z: 0.00399772
EKF Linear Acceleration 99th Percentile:
 X: 0.00422018;    Y: 0.00370575;    Z: 0.00559291
EKF Linear Acceleration Max Deviation:
 X: 0.00700350;    Y: 0.00560087;    Z: 0.00843016
EKF Linear Acceleration RMS:
 X: 0.00184113;    Y: 0.00164473;    Z: 0.00239222
EKF Linear Acceleration Covariance Matrix (rows/cols: X, Y, Z):
         X           Y           Z
X  3.3905e-06  1.1689e-06  1.7803e-06
Y  1.1689e-06  2.7057e-06  5.8046e-07
Z  1.7803e-06  5.8046e-07  5.7239e-06


In [9]:
# --- Angular Velocity Analysis ---
# Given that the Quad is static, we can analyze the angular velocity estimates as they were position measurements.
ekf_ang_vel_stats = au.position_analysis(state_estimates.velocity_angular.to_numpy())
au.print_position_analysis(ekf_ang_vel_stats, label="EKF Angular Velocity")

print(f"Maximum body rate deviation (deg/s): \n X: {np.rad2deg(ekf_ang_vel_stats.max_deviation[0]):.5f};   Y: {np.rad2deg(ekf_ang_vel_stats.max_deviation[1]):.5f};   Z: {np.rad2deg(ekf_ang_vel_stats.max_deviation[2]):.5f};" )

EKF Angular Velocity Mean:
 X: -0.0000;    Y: 0.0001;    Z: -0.0000
EKF Angular Velocity Std Dev:
 X: 0.00475065;    Y: 0.00592231;    Z: 0.00270282
EKF Angular Velocity 95th Percentile:
 X: 0.00737947;    Y: 0.01134479;    Z: 0.00437854
EKF Angular Velocity 99th Percentile:
 X: 0.00990502;    Y: 0.01489149;    Z: 0.00590201
EKF Angular Velocity Max Deviation:
 X: 0.01697227;    Y: 0.01912344;    Z: 0.01032466
EKF Angular Velocity RMS:
 X: 0.00475065;    Y: 0.00592231;    Z: 0.00270282
EKF Angular Velocity Covariance Matrix (rows/cols: X, Y, Z):
         X           Y           Z
X  2.2574e-05  -2.2388e-05  5.7833e-07
Y  -2.2388e-05  3.5081e-05  -6.5863e-06
Z  5.7833e-07  -6.5863e-06  7.3068e-06
Maximum body rate deviation (deg/s): 
 X: 0.97244;   Y: 1.09569;   Z: 0.59156;


# Pseudo-Innovation Analysis
- Using the EKF to Vicon residuals

In [11]:
# ekf_to_vicon_positions = state_estimates.position.to_numpy() - vicon_measurements.position.to_numpy()
# ekf_to_vicon_R = R.from_quat(state_estimates.orientation, scalar_first=True)*R.from_quat(vicon_measurements.orientation, scalar_first=True)

# --- Position Analysis ---
# ekf_stats = au.position_analysis(state_estimates.position.to_numpy())
# au.print_position_analysis(ekf_stats, label="EKF Position")

au.interpolate_position(vicon_measurements.position.to_numpy(), vicon_measurements.time.to_numpy(), state_estimates.time.to_numpy())



AssertionError: Original timestamps must be sorted.

# Vicon Noise Characteristics

In [12]:
acf_x = au.compute_acf(vicon_measurements.position.to_numpy()[:,0])
acf_y = au.compute_acf(vicon_measurements.position.to_numpy()[:,1])
acf_z = au.compute_acf(vicon_measurements.position.to_numpy()[:,2])

# pu.plot_acf_plotly(acf_x, title="X ACF with 95% bounds")
au.print_acf_analysis(acf_x)


=== ACF Analysis Summary ===
95% confidence band under white-noise null: [-0.0158, 0.0158]
Total lags checked: 2000 (excluding lag=0)
Significant lags: 838

Top 10 significant lags:
  Lag   0: ACF=+1.0000
  Lag   1: ACF=+0.2624
  Lag   3: ACF=+0.2166
  Lag   2: ACF=+0.2157
  Lag   4: ACF=+0.2100
  Lag  46: ACF=+0.1923
  Lag   5: ACF=+0.1922
  Lag   6: ACF=+0.1799
  Lag   7: ACF=+0.1577
  Lag   8: ACF=+0.1529


# EKF Noise Characteristics

In [13]:
acf_x = au.compute_acf(state_estimates.position.to_numpy()[:,0])
acf_y = au.compute_acf(state_estimates.position.to_numpy()[:,1])
acf_z = au.compute_acf(state_estimates.position.to_numpy()[:,2])

# pu.plot_acf_plotly(acf_x, title="X ACF with 95% bounds")

au.print_acf_analysis(acf_x)



=== ACF Analysis Summary ===
95% confidence band under white-noise null: [-0.0287, 0.0287]
Total lags checked: 1165 (excluding lag=0)
Significant lags: 528

Top 10 significant lags:
  Lag   0: ACF=+1.0000
  Lag   1: ACF=+0.5830
  Lag   2: ACF=+0.3010
  Lag  17: ACF=+0.2381
  Lag  51: ACF=+0.2378
  Lag  18: ACF=+0.2240
  Lag  19: ACF=+0.2010
  Lag  50: ACF=+0.1888
  Lag  52: ACF=+0.1872
  Lag  16: ACF=+0.1776
